In [ ]:
import os
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import pandas as pd
from utils import plot_features, calculate_performance_split, prob_to_predictlabel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
5

In [ ]:

def test(test_loader, model):
    # switch to eval mode
    model.eval()

    all_features, all_labels = [], []
    ACC = 0
    AUC = 0

    for batch_idx, data in enumerate(test_loader):
        inputs, labels = data[0].to(device), data[1].to(device)


        with torch.no_grad():
            features, outputs= model(inputs)

        all_features.append(features.data.cpu().numpy())
        all_labels.append(labels.data.cpu().numpy())


        class_probs = []
        class_label = []

        with torch.no_grad():
            class_probs_batch = [F.softmax(el, dim=0) for el in
                                 outputs]  # [tensor([0.7962, 0.2038], device='cuda:0', dtype=torch.float64)
            class_probs.append(class_probs_batch)
            class_label.append(labels)

        test_probs = torch.cat([torch.stack(batch) for batch in class_probs])

        test_label = torch.cat(class_label)


        y_pred, y_probs = prob_to_predictlabel(test_probs, test_label)

        ACC, AUC, TAR_FAR_E1, TAR_FAR_E2, TAR_FAR_E3, TAR_FAR_E0_3, FPR, TPR, thresholds = calculate_performance_split(
            test_label.cpu(), y_pred, y_probs, 1)

        print("Test Acc is : " + str(ACC))
        print("Test AUC is : " + str(AUC))
        print("TAR@FAR_E1 is : " + str(TAR_FAR_E1))
        print("TAR_FAR_E2 is : " + str(TAR_FAR_E2))
        print("TAR@FAR_E3 is : " + str(TAR_FAR_E3))
        print("TAR@FAR=0.3 is : " + str(TAR_FAR_E0_3))

    all_features = np.concatenate(all_features, 0)
    all_labels = np.concatenate(all_labels, 0)
    plot_features(all_features, all_labels, 2, isTest=True)

    if not os.path.exists('./plotdata/'):
        os.makedirs('./plotdata/')
    df_feature = pd.DataFrame(columns=('x_0', 'y_0', 'x_1', 'y_1'))
    df_feature['x_0'] = all_features[all_labels == 0, 0]
    df_feature['y_0'] = all_features[all_labels == 0, 1]
    df_feature['x_1'] = all_features[all_labels == 1, 0]
    df_feature['y_1'] = all_features[all_labels == 1, 1]
    df_feature.to_csv('./plotdata/' + 'test.csv', index=True, index_label=True)

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Resize, Compose
data_transform = Compose([Resize((20, 40)),
                          # Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                          ToTensor()])

test_data = ImageFolder(root="/root/run/testdata/",
                        transform=data_transform)

test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=len(test_data),
                                          num_workers=args.num_workers,
                                          shuffle=False)

model = torch.load('./model/model')
model.to(device)

test(test_loader, model)